<a href="https://colab.research.google.com/github/erl-j/enfusion/blob/colab-demo/enfusion_text_2_synth_sound_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://storage.googleapis.com/a-chill-bucket/enfusion_single_note_large_epoch%3D4999-step%3D110000.ckpt
!wget https://storage.googleapis.com/a-chill-bucket/enfusion/demo_assets/drums/epoch=27058-step=460000.ckpt

--2023-01-24 05:16:21--  https://storage.googleapis.com/a-chill-bucket/enfusion_single_note_large_epoch%3D4999-step%3D110000.ckpt
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.107.128, 74.125.196.128, 74.125.134.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.107.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 263692701 (251M) [application/octet-stream]
Saving to: ‘enfusion_single_note_large_epoch=4999-step=110000.ckpt.2’

enfusion_single_not 100%[===================>] 251.48M  26.5MB/s    in 10s     

2023-01-24 05:16:32 (24.6 MB/s) - ‘enfusion_single_note_large_epoch=4999-step=110000.ckpt.2’ saved [263692701/263692701]

--2023-01-24 05:16:32--  https://storage.googleapis.com/a-chill-bucket/enfusion/demo_assets/drums/epoch=27058-step=460000.ckpt
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.107.128, 74.125.196.128, 74.125.134.128, ...
Connecting to storage.googleapis.com (storage

In [2]:
! mkdir demo_assets
! mkdir demo_assets/single_note_large
! cp "enfusion_single_note_large_epoch=4999-step=110000.ckpt" demo_assets/single_note_large/epoch=4999-step=110000.ckpt

! mkdir demo_assets/drums
! cp "enfusion_single_note_large_epoch=4999-step=110000.ckpt" demo_assets/drums/epoch=27058-step=460000.ckpt


mkdir: cannot create directory ‘demo_assets’: File exists
mkdir: cannot create directory ‘demo_assets/single_note_large’: File exists
mkdir: cannot create directory ‘demo_assets/drums’: File exists


In [3]:
!git clone https://github.com/erl-j/enfusion.git

# %%
!git clone --recursive https://github.com/crowsonkb/v-diffusion-pytorch
!pip install sample-generator
!pip install v-diffusion-pytorch
!pip install ipywidgets==7.7.1 gradio
!pip install k-diffusion

!pillow==9.3.0 pip install einops diffusers wandb noisereduce prefigure v-diffusion-pytorch transformers k-diffusion encodec

fatal: destination path 'enfusion' already exists and is not an empty directory.
fatal: destination path 'v-diffusion-pytorch' already exists and is not an empty directory.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement sample-generator (from versions: none)
ERROR: No matching distribution found for sample-generator
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached argparse-1.4.0-py2.py3-none-any.whl (23 kB)


In [4]:
import sys
sys.path.append("enfusion/")


In [ ]:
# %%
import os

import noisereduce as nr
from scipy.io import wavfile

# GPU 1
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import gc
import os
# %%
import sys
from copy import deepcopy
from datetime import datetime

import einops
import IPython.display as ipd
import matplotlib.pyplot as plt
import numpy as np
import torch
import torchaudio
from einops import rearrange
#@title Imports and definitions
from torch import nn, optim
from tqdm import tqdm

from encodec_processor import EncodecProcessor
from export_sfz import export_sfz
from models import MultiPitchRecurrentScore, RecurrentScore
from sampling import resample, reverse_sample, sample
from text_embedder import TextEmbedder

device ="cuda" if torch.cuda.is_available() else "cpu"

import IPython.display as ipd
import matplotlib.pyplot as plt


def plot_and_hear(audio, sr):
    display(ipd.Audio(audio.cpu().clamp(-1, 1), rate=sr))
    plt.plot(audio.cpu().t().numpy())
  
def load_to_device(path, sr):
    audio, file_sr = torchaudio.load(path)
    if sr != file_sr:
      audio = torchaudio.transforms.Resample(file_sr, sr)(audio)
    audio = audio.to(device)
    return audio

#@title Args
sample_size = 65536 
sample_rate = 48000   
latent_dim = 0             

class Object(object):
    pass

args = Object()
args.sample_size = sample_size
args.sample_rate = sample_rate
args.latent_dim = latent_dim

sampler_type = "v-iplms" #@param ["v-iplms", "k-heun", "k-dpmpp_2s_ancestral", "k-lms", "k-dpm-2", "k-dpm-fast", "k-dpm-adaptive"]

text_embedder = TextEmbedder()

models_metadata = {
  "multipitch_parallel": {"checkpoint_path":"demo_assets/multiplenotes/parallel/epoch=4210-step=400000.ckpt", "clip_s":5, "n_pitches":9},
  "multipitch_sequential": {"checkpoint_path":"demo_assets/multiplenotes/sequential/epoch=2105-step=200000.ckpt", "clip_s":5, "n_pitches":9,"hidden_size":256,"reduced_text_embedding_size":16},
  "single_pitch": {"checkpoint_path":"demo_assets/single_note/epoch=6363-step=70000.ckpt", "clip_s":5, "n_pitches":1,"hidden_size":256, "reduced_text_embedding_size":16},
  "single_pitch_large":{"checkpoint_path":"demo_assets/single_note_large/epoch=4999-step=110000.ckpt", "clip_s":5, "n_pitches":1, "hidden_size":512, "reduced_text_embedding_size":32},
  "drums":{"checkpoint_path":"demo_assets/drums/epoch=27058-step=460000.ckpt", "clip_s":1, "n_pitches":1, "hidden_size":512, "reduced_text_embedding_size":32},
}

MODEL = "drums"

model_metadata = models_metadata[MODEL]

if MODEL == "multipitch_parallel":
    denoising_model= MultiPitchRecurrentScore(n_in_channels=128,n_conditioning_channels=512, n_pitches=models_metadata[MODEL]["n_pitches"])
else:
    denoising_model= RecurrentScore(n_in_channels=128,n_conditioning_channels=512,hidden_size=model_metadata["hidden_size"],reduced_text_embedding_size=model_metadata["reduced_text_embedding_size"])

#@title Model code
class DiffusionModel(nn.Module):
    def __init__(self, global_args , denoising_model=None):
        super().__init__()
        self.diffusion = denoising_model
        self.diffusion_ema = deepcopy(self.diffusion)
        self.rng = torch.quasirandom.SobolEngine(1, scramble=True)


ENCODEC_FRAME_RATE = 150
ENCODEC_CHANELS = 128

print("Creating the model...")
model = DiffusionModel(args, denoising_model=denoising_model)
state_dict=torch.load(model_metadata["checkpoint_path"],map_location=torch.device(device))["state_dict"]

model.load_state_dict(state_dict, strict=False)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.requires_grad_(False).to(device)
print("Model created")

# # Remove non-EMA
del model.diffusion
model_fn = model.diffusion_ema

def midi_to_hz(note):
    a = 440 #frequency of A (coomon value is 440Hz)
    return (a / 32) * (2 ** ((note - 9) / 12))

encodec_processor = EncodecProcessor(48000).to(device)

def sonify(fakes):
        embeddings = fakes
        # decode here
        fakes = encodec_processor.decode_embeddings(embeddings)

        # Put the demos together
        fakes = rearrange(fakes, "b d n -> d (b n)")
        fakes = fakes / torch.max(torch.abs(fakes) + 1e-8)
        return fakes

def text2sound(text,steps,batch_size):
  text_embedding = text_embedder.embed_text(text).to(device)[None,:].repeat(batch_size,1)

  torch.cuda.empty_cache()
  gc.collect()
  noise = torch.randn([batch_size, ENCODEC_CHANELS, ENCODEC_FRAME_RATE * model_metadata["clip_s"]* model_metadata["n_pitches"] ]).to(device)
  generated = sample(model_fn, noise, text_embedding, steps, sampler_type)
  generated = sonify(generated)
  generated = generated.cpu().detach()

  audio = generated.clamp(-1, 1)
  #audio = torchaudio.functional.highpass_biquad(audio,args.sample_rate,15.0,Q=0.707)
  # audio_memap = nr.reduce_noise(y=audio.cpu().numpy()[0], sr=args.sample_rate, prop_decrease=0.5)
  # wavfile.write("artefacts/noise_reduced.wav", args.sample_rate, audio_memap)
  # audio,sr= torchaudio.load("artefacts/noise_reduced.wav")
  # convert to numpy array
  plot_and_hear(audio, args.sample_rate)
  return audio


Downloading:   0%|          | 0.00/862k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/525k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/389 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/568 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/605M [00:00<?, ?B/s]

In [ ]:
batch_size = 1
n_steps=250

audio = text2sound("Snare drum", n_steps, batch_size)

plot_and_hear(audio, 48000)